# Toy Hidden Markov Model Demonstrations

- Andrew J. Graves
- 11/11/20

In [4]:
# Import modules
import numpy as np
import pandas as pd

# Observation matrix
B = np.array([[.4, .3, .3],
              [.3, .4, .3],
              [.3, .3, .4]])

# Transition matrix
A = np.array([[.1, .5, .4],
              [.2,  0, .8],
              [ 0, .3, .7]])

# Started in state 1 with certainty
init = np.array([1, 0, 0])

# Sensor observations (where 0 = 1)
obs = np.array([0, 2, 2])

To find the most likely state after three time-points, we need to find and normalize $\alpha(h_3)$. We can solve for $\alpha(h_t)$ with the following filtering recursion: 

$$\alpha(h_t) = p(v_t|h_t)\sum_{h_t-1}p(h_t|h_t-1)\alpha(h_{t-1})$$

In [5]:
# Run the Markov filtering recursion
for idx, i in enumerate(obs):
    if idx == 0:
        alpha = B[i] * init
    else:
        alpha = B[i] * (alpha @ A)

# Compute the probability for each state after 3 time points
p_state = alpha / np.sum(alpha)
print(f'The most likely state after 3 time points is \
{np.argmax(p_state)+1} with probability {np.max(p_state)}.')

The most likely state after 3 time points is 3 with probability 0.7721518987341772.


To find the most likely path, we need to implement the Viterbi algorithm:

In [6]:
# Run the Viterbi algorithm: initialize arrays
T = len(obs)
delta = np.zeros((T, A.shape[0]))
prev = delta.copy()
S = np.zeros(T)
delta[0] = init * B[:, obs[0]]
prev[0] = np.repeat(np.nan, 3)

for t in range(1, T):
    for j in range(A.shape[0]):
        # The most likely state given our previous state at t-1
        prob = delta[t-1] * (A[:, j])
        # The probability of the most probable state given the previous state and the observation at time t
        delta[t, j] = np.max(prob) * (B[j, obs[t]])                
        # The most probable state given previous state 
        prev[t, j] = np.argmax(prob)

cols = []
# Find the most probable hidden states
for i in range(1, T+1):
    S[T-i] = np.argmax(delta[T-i])
    cols.append(f'Time {str(i)}')
# Print out the most likely path
pd.DataFrame(S.reshape([1,3]) + 1, columns=cols)

,Time 1,Time 2,Time 3
0,1.0,3.0,3.0
